## Navigators
Navigators provide the ability to drill into the result set providing a summary of the distribution.  They are particularly useful when you are interested in gathering the domain of values for a particular Property.  Navigators can be used against a specific View, used in conjunction with either a query, a filter or both.  Navigators can be simple or very powerful, but provide a very useful way to capture results in logical buckets. 

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.19.0 Refinitiv.Data.Content, 1.0.0-beta3

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

24/01/2022 11:59:59 AM:DesktopSession. (State: DesktopSession is Pending)
24/01/2022 11:59:59 AM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
24/01/2022 11:59:59 AM:DesktopSession. (State: DesktopSession is Opened)


#### Example - Property values
The most basic usage of navigators is to determine the available values within a given Property.  For example, when searching for assets, we may want to narrow down the specific country an asset is traded in.  We can determine the available countries to work with.

In [5]:
// The following will list the countries assets are listed.  We can use this output to filter based on a specific country.
var response = Search.Definition(Search.View.SearchAll).Top(0)
                                                       .Navigators("RCSCountryLeaf(buckets:20)")  // Show the top 20
                                                       .GetData();
Console.WriteLine(response.Data.Navigators["RCSCountryLeaf"]["Buckets"]);

[
  {
    "Label": "United States",
    "Count": 12760859
  },
  {
    "Label": "Eurobond",
    "Count": 4190317
  },
  {
    "Label": "China (Mainland)",
    "Count": 1250728
  },
  {
    "Label": "Colombia",
    "Count": 1017866
  },
  {
    "Label": "South Korea",
    "Count": 563935
  },
  {
    "Label": "Germany",
    "Count": 483625
  },
  {
    "Label": "Canada",
    "Count": 406767
  },
  {
    "Label": "France",
    "Count": 246619
  },
  {
    "Label": "India",
    "Count": 198069
  },
  {
    "Label": "Japan",
    "Count": 163721
  },
  {
    "Label": "Italy",
    "Count": 134978
  },
  {
    "Label": "Chile",
    "Count": 131886
  },
  {
    "Label": "Australia",
    "Count": 103792
  },
  {
    "Label": "Switzerland",
    "Count": 65643
  },
  {
    "Label": "Brazil",
    "Count": 65338
  },
  {
    "Label": "Russia",
    "Count": 61983
  },
  {
    "Label": "Spain",
    "Count": 61924
  },
  {
    "Label": "Austria",
    "Count": 58191
  },
  {
    "Label": "Norway",
    

#### Example - Vessels
If I want to perform a basic search for tankers and capture the Hull Types, I could start with the following basic query:

In [7]:
response = Search.Definition(Search.View.VesselPhysicalAssets).Query("tanker")
                                                              .Select("DocumentTitle, HullType")
                                                              .Top(100)
                                                              .GetData();
response.Data.Table

"MOZAH, Liquefied Natural Gas; LNG Tanker, NAKILAT SHIPPING QATAR LTD, South China Sea|Manila Tanker Zone",Single Hull
"UMM SLAL, LNG Tanker, NAKILAT SHIPPING QATAR LTD, South China Sea|Canton Tanker Zone",Single Hull
"BU SAMRA, LNG Tanker, NAKILAT SHIPPING QATAR LTD, West Indian Ocean|Colombo Tanker Zone",Single Hull
"AL MAYEDA, LNG Tanker, STASCO SHIP MANAGEMENT, East China Sea|Korea Tanker Zone",Single Hull
"MEKAINES, LNG Tanker, NAKILAT SHIPPING QATAR LTD, North Atlantic Ocean|Bilbao Tanker Zone",Single Hull
"AL MAFYAR, LNG Tanker, NAKILAT SHIPPING QATAR LTD, Gulf of Oman|Arabian Gulf Tanker Zone",Single Hull
"SHAGRA, Liquefied Natural Gas; LNG Tanker, QATARGAS LIQUEFIED GAS CO LTD, Gulf of Oman|Arabian Gulf Tanker Zone",Single Hull
"ZARGA, Liquefied Natural Gas; LNG Tanker, QATAR GAS TRANSPORT - NAKILAT, East China Sea|Shanghai Tanker Zone",Single Hull
"AAMIRA, Liquefied Natural Gas; LNG Tanker, STASCO SHIP MANAGEMENT, Red Sea|Yanbu Tanker Zone",Single Hull
"RASHEEDA, LNG Tanker, QATARGAS LIQUEFIED GAS CO LTD, Arabia Sea",Single Hull
"AL DAFNA, LNG Tanker, NAKILAT SHIPPING QATAR LTD. (NSQL), South China Sea|Thailand Tanker Zone",Single Hull


Here, we can see a list of the different tankers and their Hull Types. Within this collection, while I can clearly see there are 2 different Hull Types: "Single" and "Double", I'm also seeing duplicates.  Instead of going through the effort to process the above result to pull out the unique list of Hull Types, I will instead use a Navigator - which was designed to do this work for me.

Using the above query, I will apply a Navigator by selecting the 'HullType' Property.  What this does is instruct search to bucket all unique Hull Types and summarize the total found for each. To capture the results, I will need to utilize the Content Layer API call.

In [8]:
response = Search.Definition(Search.View.VesselPhysicalAssets).Query("tanker")
                                                              .Top(0)  // Only interested in Navigator details
                                                              .Navigators("HullType")
                                                              .GetData();
Console.WriteLine(response.Data.Navigators["HullType"]["Buckets"]);

[
  {
    "Label": "Double Hull",
    "Count": 16686
  },
  {
    "Label": "Single Hull",
    "Count": 5431
  }
]


#### Example - Listing of exchanges per asset category
In the following example, I'm going to list the top 3 asset types in Canada and for each, display the top 2 exchanges trading on those assets.

In [9]:
// Filter on all active (AC) assets within Canada
var filter = "AssetState eq 'AC' and RCSExchangeCountryLeaf eq 'Canada'";

// For the top 3 categories, show the top 2 exchanges
var navigators = "RCSAssetCategoryLeaf(buckets:3, sub:ExchangeName(buckets:2))"; 

response = Search.Definition(Search.View.EquityQuotes).Filter(filter)
                                                      .Navigators(navigators)
                                                      .Top(0)
                                                      .GetData();

In [10]:
// Pretty display of the listing
var rank = 1;
var exchangeRank = 1;
foreach (var asset in response.Data.Navigators["RCSAssetCategoryLeaf"]["Buckets"])
{
    Console.WriteLine($"{rank++}. {asset["Label"]}");
    foreach (var exchange in asset["ExchangeName"]["Buckets"])
        Console.WriteLine($"\t{exchangeRank++}. {exchange["Label"]} ({exchange["Count"]})");
    exchangeRank = 1;
}

1. Equity Cash Option
	1. Montreal Options Exchange (110608)
	2. Refinitiv (17007)
2. Ordinary Share
	1. Aequitas NEO-L (3472)
	2. Aequitas NEO-N (3471)
3. Equity Future Spread
	1. Montreal Exchange (6587)
	2. Montreal Options Exchange (17)


#### Example - Outstanding Government debt in the last 3 months.
In this example, I will be using some nested expressions to summarize categories within the buckets I collect.  Specifically, I'm looking for government debt ('govt') issued any time after 3-months ago using the following filter:
- "DBType eq 'govt' and IssueDate gt today-3months" (DBType identifies government debt)
    
Based on the result set, I want categorize or create buckets (top 5) based on the domicile of the issuer.  For each domicile, sum the outstanding debt and also include the maximum coupon rate for each one.
- RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)

In [11]:
// Get date N months ago
var date = DateTime.Now.AddMonths(-3);  // Last 3 months

In [12]:
// Retrieve the top 5 outstanding government debt in the last 3 months
filter = $"DbType eq 'govt' and IssueDate gt {date:yyyy-MM-dd}";
navigators = "RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)";

response = Search.Definition(Search.View.GovCorpInstruments).Filter(filter)
                                                            .Navigators(navigators)
                                                            .Top(0)
                                                            .GetData();
Console.WriteLine(response.Data.Navigators["RCSDomicileLeaf"]["Buckets"]);

[
  {
    "Label": "United States",
    "Count": 84,
    "max_CouponRate": 2.0,
    "sum_FaceOutstandingUSD": 2612737524500.0
  },
  {
    "Label": "Japan",
    "Count": 56,
    "max_CouponRate": 0.7,
    "sum_FaceOutstandingUSD": 981566649779.0
  },
  {
    "Label": "China (Mainland)",
    "Count": 38,
    "max_CouponRate": 2.89,
    "sum_FaceOutstandingUSD": 211723424088.0
  },
  {
    "Label": "Singapore",
    "Count": 47,
    "max_CouponRate": 1.25,
    "sum_FaceOutstandingUSD": 172869794457.0
  },
  {
    "Label": "Hong Kong",
    "Count": 45,
    "max_CouponRate": 3.0,
    "sum_FaceOutstandingUSD": 123165495773.0
  }
]
